<a href="https://colab.research.google.com/github/Brayancrc/deteccaodiscursodeodio/blob/main/deteccaodiscursodeodio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Modelo de detecção de discurso de ódio**

A detecção de discurso de ódio é geralmente uma tarefa de classificação de sentimentos. Então, para treinamento, um modelo que pode classificar discurso de ódio de um certo pedaço de texto pode ser alcançado treinando-o em dados que geralmente são usados ​​para classificar sentimentos. Então, para a tarefa do modelo de detecção de discurso de ódio, usarei os dados de redes sociais.

O conjunto de dados que usarei para o modelo de detecção de discurso de ódio consiste em um conjunto de teste e treinamento. O pacote de treinamento inclui uma lista de 31.962 tweets, um ID correspondente e uma tag 0 ou 1 para cada tweet. O sentimento específico que precisamos detectar neste conjunto de dados é se o tweet é ou não baseado em discurso de ódio.

Então, vamos começar com a tarefa de construir um modelo de detecção de discurso de ódio. Vou simplesmente começar lendo os conjuntos de dados usando o pacote pandas em python:

In [1]:
import pandas as pd

from google.colab import drive
drive.mount('/content/drive')

train = pd.read_csv("/content/drive/MyDrive/DataScienceModels/train.csv")
print("Training Set:"% train.columns, train.shape, len(train))
test = pd.read_csv("/content/drive/MyDrive/DataScienceModels/test.csv")
print("Test Set:"% test.columns, test.shape, len(test))

Mounted at /content/drive
Training Set: (31962, 3) 31962
Test Set: (17197, 2) 17197


**Limpeza de Dados**

Limpeza de dados é o processo de preparar dados formatados incorretamente para análise, excluindo ou modificando os dados formatados incorretamente, o que geralmente não é necessário ou útil para análise de dados, pois pode atrapalhar o processo ou fornecer resultados imprecisos. Agora, executarei o processo de limpeza de dados usando a biblioteca re em Python:

In [2]:
import re
def  clean_text(df, text_field):
    df[text_field] = df[text_field].str.lower()
    df[text_field] = df[text_field].apply(lambda elem: re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", elem))
    return df
test_clean = clean_text(test, "tweet")
train_clean = clean_text(train, "tweet")

**Manipulando dados desbalanceados para o modelo de detecção de discurso de ódio**

Se você analisar profundamente a tarefa em que estamos trabalhando com o contexto dos dados que estamos usando, verá que os tweets sobre discursos de ódio são comparativamente menores do que outros, então esta é uma situação de dados desbalanceados.

Se ajustarmos esses dados para treinar nosso modelo de detecção de discurso de ódio, o modelo não generalizará nenhum discurso de ódio porque os dados com contexto para o discurso de ódio são muito menores do que os positivos. Então, nessa situação, precisamos preparar os dados para se ajustarem adequadamente ao nosso modelo.

Há vários métodos que você pode usar para lidar com isso. Uma abordagem é usar oversampling ou downsampling. No caso de oversampling, usamos uma função que faz amostragem repetidamente, com substituição, da classe minoritária até que a classe tenha o mesmo tamanho da maioria. Vamos ver como podemos lidar com isso:

In [3]:
from sklearn.utils import resample
train_majority = train_clean[train_clean.label==0]
train_minority = train_clean[train_clean.label==1]
train_minority_upsampled = resample(train_minority,
                                 replace=True,
                                 n_samples=len(train_majority),
                                 random_state=123)
train_upsampled = pd.concat([train_minority_upsampled, train_majority])
train_upsampled['label'].value_counts()

,count
label,
1,29720
0,29720


**Criando um Pipeline**

Para simplicidade e reprodutibilidade do modelo de detecção de discurso de ódio, usei o pipeline do Scikit-Learn com um SGDClassifier, antes de treinar nosso modelo:

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier
pipeline_sgd = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf',  TfidfTransformer()),
    ('nb', SGDClassifier()),])

**Treinando o modelo de detecção de discurso de ódio**

Agora, antes de treinar o modelo, vamos dividir os dados em um conjunto de treinamento e um conjunto de teste:

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train_upsampled['tweet'],
                                                    train_upsampled['label'],random_state = 0)

Agora vamos treinar o modelo e prever os resultados no conjunto de teste usando o método de pontuação F1:

In [6]:
model = pipeline_sgd.fit(X_train, y_train)
y_predict = model.predict(X_test)
from sklearn.metrics import f1_score
f1_score(y_test, y_predict)

0.9696605987864239

Então obtivemos uma pontuação F1 de 0,96 por cento, o que é geralmente apreciável. Este modelo agora pode ser implantado e usado em produção.